# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Julia:-Numerical-Linear-Algebra" data-toc-modified-id="Julia:-Numerical-Linear-Algebra-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Julia: Numerical Linear Algebra</a></div><div class="lev2 toc-item"><a href="#Numerical-linear-algebra:-introduction" data-toc-modified-id="Numerical-linear-algebra:-introduction-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Numerical linear algebra: introduction</a></div><div class="lev2 toc-item"><a href="#BLAS" data-toc-modified-id="BLAS-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>BLAS</a></div><div class="lev2 toc-item"><a href="#Memory-hierarchy-and-level-3-fraction" data-toc-modified-id="Memory-hierarchy-and-level-3-fraction-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Memory hierarchy and level-3 fraction</a></div><div class="lev2 toc-item"><a href="#Effect-of-data-layout" data-toc-modified-id="Effect-of-data-layout-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Effect of data layout</a></div><div class="lev2 toc-item"><a href="#Avoid-memory-allocation:-some-examples" data-toc-modified-id="Avoid-memory-allocation:-some-examples-15"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Avoid memory allocation: some examples</a></div><div class="lev3 toc-item"><a href="#Transposing-matrix-is-expensive" data-toc-modified-id="Transposing-matrix-is-expensive-151"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>Transposing matrix is expensive</a></div><div class="lev2 toc-item"><a href="#Sparse-linear-algebra" data-toc-modified-id="Sparse-linear-algebra-16"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Sparse linear algebra</a></div><div class="lev2 toc-item"><a href="#Iterative-methods-for-linear-algebra" data-toc-modified-id="Iterative-methods-for-linear-algebra-17"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Iterative methods for linear algebra</a></div>

# Julia: Numerical Linear Algebra

Numerical linear algebra occupies much of statistical computing. This notebook gives a quick overview of linear algebra in Julia.

In [ ]:
versioninfo()

## Numerical linear algebra: introduction

* Numerical linear algebra: 
    - BLAS: vector operations, matrix-vector multiplications, matrix-matrix multiplications  
    - solve linear equations $\mathbf{A} \mathbf{x} = \mathbf{b}$
    - regression computations $\mathbf{X}^T \mathbf{X} \beta = \mathbf{X}^T \mathbf{y}$  
    - eigen-problems $\mathbf{A} \mathbf{x} = \lambda \mathbf{x}$  
    - generalized eigen-problems $\mathbf{A} \mathbf{x} = \lambda \mathbf{B} \mathbf{x}$  
    - singular value decompositions $\mathbf{A} = \mathbf{U} \Sigma \mathbf{V}^T$  
    - iterative methods for numerical linear algebra    

* Except for the iterative methods, most of these numerical linear algebra tasks are implemented in the BLAS and LAPACK libraries. They form the **building blocks** of most statistical computing tasks (optimization, MCMC).

* All high-level languages (R, Matlab, Julia) call BLAS and LAPACK for numerical linear algebra. 

Let's first benchmark the common matrix multiplication in Julia.

In [ ]:
using BenchmarkTools

srand(123) # seed
n = 1000
A = randn(n, n)
B = randn(n, n)
@benchmark $A * $B

How about R?

In [ ]:
using RCall

R"""
library(microbenchmark)
microbenchmark($A %*% $B)
"""

Base R is using a very outdated BLAS library. For this matrix multiplication example, we see a ~30-40 fold speedup by Julia's OpenBLAS library.

## BLAS

* BLAS stands for _basic linear algebra subprograms_. 

* See [netlib](http://www.netlib.org/blas/) for a complete list of standardized BLAS functions.

* There are many implementations of BLAS. 
    - [Netlib](http://www.netlib.org/blas/) provides a reference implementation  
    - Matlab uses Intel's [MKL](https://software.intel.com/en-us/node/520724) (Mathematical Kernel Libaries)  
    - Julia uses [OpenBLAS](https://github.com/xianyi/OpenBLAS)  
    - JuliaPro offers the option of using MKL

* There are 3 levels of BLAS functions.
    - [Level 1](http://www.netlib.org/blas/#_level_1): vector-vector operation
    - [Level 2](http://www.netlib.org/blas/#_level_2): matrix-vector operation
    - [Level 3](http://www.netlib.org/blas/#_level_3): matrix-matrix operation

| Level | Example Operation                      | Name        | Dimension                                 | Flops |
|-------|----------------------------------------|-------------|-------------------------------------------|-------|
| 1     | $\alpha \gets \mathbf{x}^T \mathbf{y}$ | dot product | $\mathbf{x}, \mathbf{y} \in \mathbb{R}^n$ | $2n$  |
|       | $\mathbf{y} \gets \mathbf{y} + \alpha \mathbf{x}$ |  axpy           |  $\alpha \in \mathbb{R}$, $\mathbf{x}, \mathbf{y} \in \mathbb{R}^n$ |  $2n$    |
| 2     | $\mathbf{y} \gets \mathbf{y} + \mathbf{A} \mathbf{x}$ |  gaxpy           |  $\mathbf{A} \in \mathbb{R}^{m \times n}$, $\mathbf{x} \in \mathbb{R}^n$, $\mathbf{y} \in \mathbb{R}^m$                                     |  $2mn$     |
|       | $\mathbf{A} \gets \mathbf{A} + \mathbf{y} \mathbf{x}^T$ | rank one update            |    $\mathbf{A} \in \mathbb{R}^{m \times n}$, $\mathbf{x} \in \mathbb{R}^n$, $\mathbf{y} \in \mathbb{R}^m$                                       | $2mn$      |
| 3     | $\mathbf{C} \gets \mathbf{C} + \mathbf{A} \mathbf{B}$                                       |  matrix multiplication           |  $\mathbf{A} \in \mathbb{R}^{m \times p}$, $\mathbf{B} \in \mathbb{R}^{p \times n}$, $\mathbf{C} \in \mathbb{R}^{m \times n}$                                         | $2mnp$      |

* Typical BLAS functions support single precision (S), double precision (D), complex (C), and double complex (Z). 

* For linear algebra functions in Float32 and Flaot64, Julia dispatches to BLAS as much as possible.

* Julia also offers the flexibility by exposing interfaces to many BLAS/LAPACK subroutines directly. See documentation: [BLAS](https://docs.julialang.org/en/stable/stdlib/linalg/#BLAS-Functions-1), [LAPACK](https://docs.julialang.org/en/stable/stdlib/linalg/#LAPACK-Functions-1).

## Memory hierarchy and level-3 fraction

> **Key to high performance is effective use of memory hierarchy. True on all architectures.**

* Flop count is not the sole determinant of algorithm efficiency. Another important factor is data movement through the memory hierarchy.

<img src="./macpro_inside.png" width="400" align="center">

<img src="./cpu_die.png" width="400" align="center">  

<img src="http://images.bit-tech.net/content_images/2007/11/the_secrets_of_pc_memory_part_1/hei.png" width="400" align="center">

* Numbers everyone should know

| Operation                           | Time           |
|-------------------------------------|----------------|
| L1 cache reference                  | 0.5 ns         |
| L2 cache reference                  | 7 ns           |
| Main memory reference               | 100 ns         |
| Read 1 MB sequentially from memory  | 250,000 ns     |
| Read 1 MB sequentially from SSD     | 1,000,000 ns   |  
| Read 1 MB sequentially from disk    | 20,000,000 ns  |


<!-- | Operation                           | Time           | -->
<!-- |-------------------------------------|----------------| -->
<!-- | L1 cache reference                  | 0.5 ns         | -->
<!-- | Branch mispredict                   | 5 ns           | -->
<!-- | L2 cache reference                  | 7 ns           | -->
<!-- | Mutex lock/unlock                   | 100 ns         | -->
<!-- | Main memory reference               | 100 ns         | -->
<!-- | Compress 1K bytes with Zippy        | 10,000 ns      | -->
<!-- | Send 2K bytes over 1 Gbps network   | 20,000 ns      | -->
<!-- | Read 1 MB sequentially from memory  | 250,000 ns     | -->
<!-- | Round trip within same datacenter   | 500,000 ns     | -->
<!-- | Disk seek                           | 10,000,000 ns  | -->
<!-- | Read 1 MB sequentially from network | 10,000,000 ns  | -->
<!-- | Read 1 MB sequentially from disk    | 30,000,000 ns  | -->
<!-- | Send packet CA->Netherlands->CA     | 150,000,000 ns | -->

   Source: <https://gist.github.com/jboner/2841832>  

* For example, Xeon X5650 CPU has a theoretical throughput of 128 DP GFLOPS but a max memory bandwidth of 32GB/s.  

* Can we keep CPU cores busy with enough deliveries of matrix data and ship the results to memory fast enough to avoid backlog?  
Answer: use **high-level BLAS** as much as possible.

| BLAS                                                           | Dimension                                                                           | Mem. Refs. | Flops  | Ratio |
|----------------------------------------------------------------|-------------------------------------------------------------------------------------|------------|--------|-------|
| Level 1: $\mathbf{y} \gets \mathbf{y} + \alpha \mathbf{x}$     | $\mathbf{x}, \mathbf{y} \in \mathbb{R}^n$                                           | $3n$       | $2n$   | 3:2   |
| Level 2: $\mathbf{y} \gets \mathbf{y} + \mathbf{A} \mathbf{x}$ | $\mathbf{x}, \mathbf{y} \in \mathbb{R}^n$, $\mathbf{A} \in \mathbb{R}^{n \times n}$ | $n^2$      | $2n^2$ | 1:2   |
| Level 3: $\mathbf{C} \gets \mathbf{C} + \mathbf{A} \mathbf{B}$ | $\mathbf{A}, \mathbf{B}, \mathbf{C} \in\mathbb{R}^{n \times n}$                    | $4n^2$     | $2n^3$ | 2:n |  

* Higher level BLAS (3 or 2) make more effective use of arithmetic logic units (ALU) by keeping them busy. **Surface-to-volume** effect.  
See [Dongarra slides](https://www.samsi.info/wp-content/uploads/2017/02/SAMSI-0217_Dongarra.pdf).

<img src="./blas_throughput.png" width="500" align="center"/>

* A distinction between LAPACK and LINPACK (older version of R uses LINPACK) is that LAPACK makes use of higher level BLAS as much as possible (usually by smart partitioning) to increase the so-called **level-3 fraction**.

## Effect of data layout

* Data layout in memory affects algorithmic efficiency too. It is much faster to move chunks of data in memory than retrieving/writing scattered data.

* Storage mode: **column-major** (Fortran, Matlab, R, Julia) vs **row-major** (C/C++).

* **Cache line** is the minimum amount of cache which can be loaded and stored to memory.
    - x86 CPUs: 64 bytes  
    - ARM CPUS: 32 bytes

<img src="https://patterns.eecs.berkeley.edu/wordpress/wp-content/uploads/2013/04/dense02.png" width="500" align="center"/>

* Accessing column-major stored matrix by rows causes lots of **cache misses**.

* Take matrix multiplication as an example 
$$ 
\mathbf{C} \gets \mathbf{C} + \mathbf{A} \mathbf{B}, \quad \mathbf{A} \in \mathbb{R}^{m \times p}, \mathbf{B} \in \mathbb{R}^{p \times n}, \mathbf{C} \in \mathbb{R}^{m \times n}.
$$
Assume the storage is column-major, such as in Julia. There are 6 variants of the algorithms according to the order in the triple loops. 
    - `jki` or `kji` looping:
        ```julia
        # inner most loop
        for i = 1:m
            C[i, j] = C[i, j] + A[i, k] * B[k, j]
        end
        ```  
    - `ikj` or `kij` looping:
        ```julia
        # inner most loop        
        for j = 1:n
            C[i, j] = C[i, j] + A[i, k] * B[k, j]
        end
        ```  
    - `ijk` or `jik` looping:
        ```julia
        # inner most loop        
        for k = 1:p
            C[i, j] = C[i, j] + A[i, k] * B[k, j]
        end
        ```
* We pay attention to the innermost loop, where the vector calculation occurs. The associated **stride** when accessing the three matrices in memory (assuming column-major storage) is  

| Variant        | A Stride | B Stride | C Stride |
|----------------|----------|----------|----------|
| $jki$ or $kji$ | Unit     | 0        | Unit     |
| $ikj$ or $kij$ | 0        | Non-Unit | Non-Unit |
| $ijk$ or $jik$ | Non-Unit | Unit     | 0        |       
Apparently the variants $jki$ or $kji$ are preferred.

In [ ]:
"""
    matmul_by_loop!(A, B, C, order)

Overwrite `C` by `A * B`. `order` indicates the looping order for triple loop.
"""
function matmul_by_loop!(A::Matrix, B::Matrix, C::Matrix, order::String)
    
    m = size(A, 1)
    p = size(A, 2)
    n = size(B, 2)
    fill!(C, 0)
    
    if order == "jki"
        for j = 1:n, k = 1:p, i = 1:m
            C[i, j] += A[i, k] * B[k, j]
        end
    end

    if order == "kji"
        for k = 1:p, j = 1:n, i = 1:m
            C[i, j] += A[i, k] * B[k, j]
        end
    end
    
    if order == "ikj"
        for i = 1:m, k = 1:p, j = 1:n
            C[i, j] += A[i, k] * B[k, j]
        end
    end

    if order == "kij"
        for k = 1:p, i = 1:m, j = 1:n
            C[i, j] += A[i, k] * B[k, j]
        end
    end
    
    if order == "ijk"
        for i = 1:m, j = 1:n, k = 1:p
            C[i, j] += A[i, k] * B[k, j]
        end
    end
    
    if order == "jik"
        for j = 1:n, i = 1:m, k = 1:p
            C[i, j] += A[i, k] * B[k, j]
        end
    end
    
end

srand(123) # seed
m, n, p = 2000, 100, 2000
A = rand(m, n)
B = rand(n, p)
C = zeros(m, p);

* $jki$ and $kji$ looping:

In [ ]:
using BenchmarkTools

@benchmark matmul_by_loop!($A, $B, $C, "jki")

In [ ]:
@benchmark matmul_by_loop!($A, $B, $C, "kji")

* $ikj$ and $kij$ looping:

In [ ]:
@benchmark matmul_by_loop!($A, $B, $C, "ikj")

In [ ]:
@benchmark matmul_by_loop!($A, $B, $C, "kij")

* $ijk$ and $jik$ looping:

In [ ]:
@benchmark matmul_by_loop!($A, $B, $C, "ijk")

In [ ]:
@benchmark matmul_by_loop!($A, $B, $C, "ijk")

* Julia wraps BLAS library for matrix multiplication. We see BLAS library wins hands down (multi-threading, Strassen algorithm, higher level-3 fraction by block outer product).

In [ ]:
@benchmark A_mul_B!($C, $A, $B)

In [ ]:
@benchmark Base.LinAlg.BLAS.gemm!('N', 'N', 1.0, $A, $B, 1.0, $C)

To appreciate the efforts in an optimized BLAS implementation such as OpenBLAS (evolved from GotoBLAS), see the [Quora question](https://www.quora.com/What-algorithm-does-BLAS-use-for-matrix-multiplication-Of-all-the-considerations-e-g-cache-popular-instruction-sets-Big-O-etc-which-one-turned-out-to-be-the-primary-bottleneck), especially the [video](https://youtu.be/JzNpKDW07rw). Bottomline is 

> **Get familiar with (good implementations of) BLAS/LAPACK and use them as much as possible.**

## Avoid memory allocation: some examples

### Transposing matrix is expensive

* In R, the command 
    ```R
    t(A) %*% x
    ```
will first transpose `A` then perform matrix multiplication, causing unnecessary memory allocation
- Julia is smart to avoid transposing matrix if possible.

In [ ]:
srand(123)

n = 1000
A = rand(n, n)
x = rand(n)

# dispatch to At_mul_B (and then to BLAS)
# does *not* actually transpose the matrix
@benchmark $A' * $x

In [ ]:
@which A' * x

In [ ]:
# dispatch to BLAS
@benchmark At_mul_B($A, $x)

In [ ]:
# let's force transpose
@benchmark transpose($A) * $x

In [ ]:
# pre-allocate result
out = zeros(size(A, 2))
@benchmark At_mul_B!($out, $A, $x)

In [ ]:
using RCall

R"""
library(microbenchmark)
microbenchmark(t($A) %*% $x)
"""

* [Broadcasting](https://docs.julialang.org/en/stable/manual/functions/#man-vectorized-1) in Julia achieves vectorized code without creating intermediate arrays.

In [ ]:
srand(123)
X, Y = rand(1000,1000), rand(1000,1000)

# two temporary arrays are created
@benchmark max(abs(X), abs(Y))

In [ ]:
# no temporary arrays created
@benchmark max.(abs.(X), abs.(Y))

In [ ]:
# no memory allocation at all!
Z = zeros(X)
@benchmark $Z .= max.(abs.($X), abs.($Y))

* [View](https://docs.julialang.org/en/stable/stdlib/arrays/#Base.view) avoids creating extra copy of matrix data.

In [ ]:
srand(123) # seed
A = randn(1000, 1000)

# sum entries in a sub-matrix
@benchmark sum($A[1:2:500, 1:2:500])

In [ ]:
# view avoids creating a separate sub-matrix
@benchmark sum(@view $A[1:2:500, 1:2:500])

## Sparse linear algebra

Julia has native support for sparse linear algebra.

Generate an n-by-n random sparse matrix

In [ ]:
srand(123) # seed
n, p = 1000, 500
A = sprandn(n, p, 0.001) # about 0.1% non-zeros

For comparison we create the corresponding dense matrix

In [ ]:
Afull = full(A)

How much memory does sparse A take?

In [ ]:
Base.summarysize(A) 

How much memory does dense A take?

In [ ]:
Base.summarysize(Afull)

Matrix-vector multiplication:

In [ ]:
b = randn(p)
@benchmark $A * $b # sparse linear algebra

In [ ]:
@benchmark $Afull * $b # dense linear algebra, i.e., BLAS

Least squares problem:

In [ ]:
y = randn(n)
@benchmark $A \ $y # sparse linear algebra

In [ ]:
@benchmark $Afull \ $y # dense linear algebra, i.e., LAPACK

## Iterative methods for linear algebra

Singular value decomposition (SVD)

In [ ]:
@benchmark svds(A) # Lanczos iterative method for top singular values/vectors

In [ ]:
@benchmark svd(Afull) # dense linear algebra, i.e., LAPACK

[IterativeSolvers.jl](https://github.com/JuliaMath/IterativeSolvers.jl) package implements many common iterative methods for sparse or more general structured matrices: conjugate gradient (CG), LSQR/LSMR for least squares, ... Combined with [LinearMaps.jl](https://github.com/Jutho/LinearMaps.jl) package, it provides powerful numerial linear algebra engine for structured large arrays.